In [8]:
# libararies
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import copy as copy
import os


MATCH KEYPOINTS FUNCTION

In [9]:
def matchKeyPoints(keyPoints1, keyPoints2, descriptors1, descriptors2):
    
    matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
    
    descriptors1 = np.float32(descriptors1)
    descriptors2 = np.float32(descriptors2)
    
    raw_matches = matcher.knnMatch(descriptors1, descriptors2, k=2)
    
    matches = []
    for m in raw_matches:
        if len(m) == 2 and m[0].distance < m[1].distance * 0.75:
            matches.append((m[0].trainIdx, m[0].queryIdx))
    
    if len(matches) > 4:
        
        keyPoints1 = np.float32([keyPoints1[i] for (_, i) in matches])
        keyPoints2 = np.float32([keyPoints2[i] for (i, _) in matches])
        
        H, status = cv.findHomography(keyPoints1, keyPoints2, cv.RANSAC, 4.0)
        
            
    else:
        H, status = None, None
    
    return matches, H, status

WIDTH AND HEIGHT DIMENSION FINDING FUNCTION

In [10]:
def findImgWidthDim(Img):
    min_idx_width = 10000
    max_idx_width = 0
    min_idx_height = 10000
    max_idx_height = 0

    
    
    height, width, _ = Img.shape

    threshold = 120

    for idx in range(width):
        if Img[350, idx, 0] + Img[350, idx, 1] + Img[350, idx, 2] < threshold:
            min_idx_width = idx + 1
        else:
            break
        
    for idx in range(width):
        if Img[350, width - idx - 1, 0] + Img[350, width - idx - 1, 1] + Img[350, width - idx - 1, 2] < threshold:
            max_idx_width = width - idx - 2
        else:
            break
        
    for idx in range(height):
        if Img[idx, 350, 0] + Img[idx, 350, 1] + Img[idx, 350, 2] < threshold:
            min_idx_height = idx + 1
        else:
            break
        
    for idx in range(height):
        if Img[height - idx - 1, 350, 0] + Img[height - idx - 1, 350, 1] + Img[height - idx - 1, 350, 2] < threshold:
            max_idx_height = width - idx - 2
        else:
            break
    
    print('at width, min: %d, max: %d' % (min_idx_width, max_idx_width))
    print('at height, min: %d, max: %d' % (min_idx_height, max_idx_height))
    return min_idx_width, max_idx_width, min_idx_height, max_idx_height 

1. IMPORT AND VISUALIZE IMAGES

In [11]:
# recall color and gray images and mask
color_ref = cv.imread('./PlanarImages/reference/Ref.jpg')
gray_ref = cv.cvtColor(color_ref, cv.COLOR_BGR2GRAY)

path_colors = './PlanarImages/image/'
path_masks = './PlanarImages/mask/'
colors = []
grays = []
masks = []

for root, directions, files in os.walk(path_colors):
    
    for file in files:
        if '.jpg' in file:
            
            img_input = cv.imread(os.path.join(root, file))
            gray_input = cv.cvtColor(img_input, cv.COLOR_BGR2GRAY)
            
            colors.append(img_input)
            grays.append(gray_input)

for root, directions, files in os.walk(path_masks):
    
    for file in files:
        if '.png' in file:
            
            mask = cv.imread(os.path.join(root, file))
            masks.append(mask)
            

2. EXTRACT FEATURES

In [12]:
# feature detector
detector = cv.BRISK_create()
key_pts_ref, descriptors_ref = detector.detectAndCompute(gray_ref, None)
key_pts_ref = np.float32([key_pt.pt for key_pt in key_pts_ref])

    # referenece image with buffer
buffer = 200
height_ref, width_ref, channel_ref = color_ref.shape
color_ref_buffer = np.zeros((height_ref + 2*buffer, width_ref + 2*buffer, channel_ref), dtype=np.uint8)
color_ref_buffer[buffer-1:height_ref + buffer-1, buffer-1:width_ref + buffer-1] = color_ref

gray_ref_buffer = cv.cvtColor(color_ref_buffer, cv.COLOR_BGR2GRAY)
key_pts_ref_buffer, descriptors_ref_buffer = detector.detectAndCompute(gray_ref_buffer, None)
key_pts_ref_buffer = np.float32([key_pt.pt for key_pt in key_pts_ref_buffer])


key_pts = []
descriptors = []

for i in range(4):
    
    key_pt, descriptor = detector.detectAndCompute(grays[i], None)
    key_pt = np.float32([k_pt.pt for k_pt in key_pt])
    key_pts.append(key_pt)
    descriptors.append(descriptor)


3. MATCHING w/ RANSAC

In [13]:
height_ref_buffer, width_ref_buffer, _ = color_ref_buffer.shape

img_combined = color_ref_buffer.copy()
min_idx_width, max_idx_width, min_idx_height, max_idx_height = findImgWidthDim(img_combined)

for i in range(4):

    matches, H, status = matchKeyPoints(key_pts[i], key_pts_ref_buffer, descriptors[i], descriptors_ref_buffer)
    result = cv.warpPerspective(colors[i], H, (width_ref_buffer, height_ref_buffer))
    result[min_idx_height:max_idx_height, min_idx_width:max_idx_width] = img_combined[min_idx_height:max_idx_height, min_idx_width:max_idx_width]
    img_combined = result.copy()
    min_idx_width, max_idx_width, min_idx_height, max_idx_height = findImgWidthDim(img_combined)



cv.imwrite('./Result.png', img_combined)


at width, min: 201, max: 1105
at height, min: 208, max: 1104


/home/juyeop/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in ubyte_scalars
  
/home/juyeop/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in ubyte_scalars
/home/juyeop/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in ubyte_scalars
/home/juyeop/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: overflow encountered in ubyte_scalars


at width, min: 102, max: 1104
at height, min: 200, max: 1105
at width, min: 6, max: 1102
at height, min: 200, max: 1105
at width, min: 6, max: 1286
at height, min: 200, max: 1105
at width, min: 6, max: 1285
at height, min: 199, max: 1105


True

In [14]:
H_set = []
f = open('./TP1_homography.txt','w')
for i in range(4):
    key_pts_image = key_pts[i].copy()
    descriptors_image = descriptors[i].copy()
    matches, H, status = matchKeyPoints(key_pts_image, key_pts_ref, descriptors_image, descriptors_ref)
    H = np.reshape(H, (1, 9))
    
    for j in range(9):
        f.write('%f '% H[0,j])
    f.write('\n')
    
    H_set.append(H)

f.close()
